In [ ]:
import pandas as pd

from rp2 import get_data_path

In [ ]:
phagocyte_genes_df = pd.read_excel(
    get_data_path("hagai_2018", "41586_2018_657_MOESM4_ESM.xlsx"),
    sheet_name="phagocytes_FC_diveregnce"
)
display(phagocyte_genes_df)

In [ ]:
responsive_phagocyte_genes = phagocyte_genes_df.loc[phagocyte_genes_df.mouse_padj < 0.01].gene
print(f"{len(responsive_phagocyte_genes):,} responsive phagocyte genes")